## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
import pandas as pd
import os
import numpy as np

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [2]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [3]:
df['date'] = pd.to_datetime(df['timestamp']).dt.date
df['Time'] = pd.to_datetime(df['timestamp']).dt.time
df['Year'] = pd.to_datetime(df['timestamp']).dt.year
df['Month'] = pd.to_datetime(df['timestamp']).dt.month
df['Day'] = pd.to_datetime(df['timestamp']).dt.day

In [4]:
df= df.drop(columns='ID')
df=df.drop(columns='timestamp')

In [5]:
df['Bump'] = [int(d) for d in df['Bump']]
df['Crossing'] = [int(d) for d in df['Crossing']]
df['Give_Way'] = [int(d) for d in df['Give_Way']]
df['Junction'] = [int(d) for d in df['Junction']]
df['No_Exit'] = [int(d) for d in df['No_Exit']]
df['Railway'] = [int(d) for d in df['Railway']]
df['Roundabout'] = [int(d) for d in df['Roundabout']]
df['Stop'] = [int(d) for d in df['Stop']]
df['Amenity'] = [int(d) for d in df['Amenity']]
df['Side'] = [x if x != "L" else 0 for x in df['Side']]
df['Side'] = [x if x != "R" else 1 for x in df['Side']] 


In [6]:
df1 = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))

print("The shape of the dataset is {}.\n\n".format(df1.shape))
df1=df1.drop_duplicates(['Year','Month','Day'],keep="last")

The shape of the dataset is (6901, 12).




In [7]:
import xml.etree.ElementTree as et
xtree = et.parse(os.path.join(dataset_path, "holidays.xml"))
xroot = xtree.getroot()
df_cols = ["date", "description"]
rows = []

for node in xroot: 
    s_description = node.find("description").text 
    s_date = node.find("date").text 
    
    
    rows.append({"description": s_description, "date": s_date})

df2 = pd.DataFrame(rows, columns = df_cols)
df2['date'] = pd.to_datetime(df2['date']).dt.date

In [8]:
df5=pd.merge(df,df2,on="date",how="left")


In [9]:
df5['description'] = [0 if x != "" else 1 for x in df5['description']]

In [10]:
df5=pd.merge(df5,df1,on=["Year","Month","Day"],how="left")

In [11]:
df5['Wind_Chill(F)'] = df5['Wind_Chill(F)'].fillna(np.mean(df5['Wind_Chill(F)']))
df5['Precipitation(in)'] = df5['Precipitation(in)'].fillna(np.mean(df5['Precipitation(in)']))
df5['Temperature(F)'] = df5['Temperature(F)'].fillna(np.mean(df5['Temperature(F)']))
df5['Humidity(%)'] = df5['Humidity(%)'].fillna(np.mean(df5['Humidity(%)']))
df5['Wind_Speed(mph)'] = df5['Wind_Speed(mph)'].fillna(np.mean(df5['Wind_Speed(mph)']))
df5['Visibility(mi)'] = df5['Visibility(mi)'].fillna(np.mean(df5['Visibility(mi)']))
df5['Weather_Condition'] = df5['Weather_Condition'].fillna(df5['Weather_Condition'].value_counts().index[0])

In [12]:
WC=pd.get_dummies(df5['Weather_Condition'])
df5 = pd.concat([df5, WC],axis=1)

In [13]:
df5 = df5.drop(columns=['Weather_Condition','date' , 'Time' , 'Selected'])

In [14]:
from sklearn.model_selection import train_test_split
from boruta import BorutaPy

train_df, val_df = train_test_split(df5, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['Severity'])
y_val = val_df['Severity']

from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

br_sel = BorutaPy(classifier , n_estimators='auto' , verbose=2 , random_state=1)
br_sel.fit(np.array(X_train),np.array(y_train))

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	46
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	46
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	46
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	46
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	46
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	46
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	46
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	14
Tentative: 	7
Rejected: 	25
Iteration: 	9 / 100
Confirmed: 	14
Tentative: 	7
Rejected: 	25
Iteration: 	10 / 100
Confirmed: 	14
Tentative: 	7
Rejected: 	25
Iteration: 	11 / 100
Confirmed: 	14
Tentative: 	7
Rejected: 	25
Iteration: 	12 / 100
Confirmed: 	15
Tentative: 	6
Rejected: 	25
Iteration: 	13 / 100
Confirmed: 	15
Tentative: 	5
Rejected: 	26
Iteration: 	14 / 100
Confirmed: 	15
Tentative: 	5
Rejected: 	26
Iteration: 	15 / 100
Confirmed: 	15
Tentative: 	5
Rejected: 	26
Iteration: 	16 / 100
Confirmed: 	15
Tentative: 	5
Reject

BorutaPy(estimator=RandomForestClassifier(max_depth=2, n_estimators=291,
                                          random_state=RandomState(MT19937) at 0x7F35574E56B0),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7F35574E56B0, verbose=2)

In [15]:
print(br_sel.support_)

[ True  True False  True  True False False False False False  True  True
  True  True  True False False False  True  True False  True  True False
 False False False  True False False False False False False False False
 False False False  True False False False False False False]


In [16]:
print(br_sel.n_features_)

15


In [17]:
sel_fe = pd.DataFrame({'Fet':list(X_train.columns),"Ran":br_sel.ranking_})
sel_fe.sort_values(by="Ran")

,Fet,Ran
0,Lat,1
39,Partly Cloudy,1
27,Fair,1
21,Humidity(%),1
19,Precipitation(in),1
18,Wind_Chill(F),1
14,Month,1
13,Year,1
12,Side,1
11,Amenity,1


The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [18]:
X_train = X_train[['Lat','Lng','Partly Cloudy','Distance(mi)','Crossing','Fair','Wind_Speed(mph)','Humidity(%)','Precipitation(in)','Stop','Amenity','Side','Year','Month','Wind_Chill(F)']]
X_val = X_val[['Lat','Lng','Partly Cloudy','Distance(mi)','Crossing','Fair','Wind_Speed(mph)','Humidity(%)','Precipitation(in)','Stop','Amenity','Side','Year','Month','Wind_Chill(F)']]


As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [19]:
classifier = classifier.fit(X_train, y_train)
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.7472698907956318


Now let's test our classifier on the validation dataset and see the accuracy.

Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [20]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [21]:
test_df['date'] = pd.to_datetime(test_df['timestamp']).dt.date
test_df['Time'] = pd.to_datetime(test_df['timestamp']).dt.time
test_df['Year'] = pd.to_datetime(test_df['timestamp']).dt.year
test_df['Month'] = pd.to_datetime(test_df['timestamp']).dt.month
test_df['Day'] = pd.to_datetime(test_df['timestamp']).dt.day

In [22]:
test_df=test_df.drop(columns='timestamp')

In [23]:
test_df['Bump'] = [int(d) for d in test_df['Bump']]
test_df['Crossing'] = [int(d) for d in test_df['Crossing']]
test_df['Give_Way'] = [int(d) for d in test_df['Give_Way']]
test_df['Junction'] = [int(d) for d in test_df['Junction']]
test_df['No_Exit'] = [int(d) for d in test_df['No_Exit']]
test_df['Railway'] = [int(d) for d in test_df['Railway']]
test_df['Roundabout'] = [int(d) for d in test_df['Roundabout']]
test_df['Stop'] = [int(d) for d in test_df['Stop']]
test_df['Amenity'] = [int(d) for d in test_df['Amenity']]
test_df['Side'] = [x if x != "L" else 0 for x in test_df['Side']]
test_df['Side'] = [x if x != "R" else 1 for x in test_df['Side']] 


In [24]:
test_df5=pd.merge(test_df,df2,on="date",how="left")
test_df5['description'] = [0 if x != "" else 1 for x in test_df5['description']]

In [25]:
test_df5=pd.merge(test_df5,df1,on=["Year","Month","Day"],how="left")

In [26]:
test_df5['Wind_Chill(F)'] = test_df5['Wind_Chill(F)'].fillna(np.mean(test_df5['Wind_Chill(F)']))
test_df5['Precipitation(in)'] = test_df5['Precipitation(in)'].fillna(np.mean(test_df5['Precipitation(in)']))
test_df5['Temperature(F)'] = test_df5['Temperature(F)'].fillna(np.mean(test_df5['Temperature(F)']))
test_df5['Humidity(%)'] = test_df5['Humidity(%)'].fillna(np.mean(test_df5['Humidity(%)']))
test_df5['Wind_Speed(mph)'] = test_df5['Wind_Speed(mph)'].fillna(np.mean(test_df5['Wind_Speed(mph)']))
test_df5['Visibility(mi)'] = test_df5['Visibility(mi)'].fillna(np.mean(test_df5['Visibility(mi)']))

In [27]:
test_df5['Weather_Condition'] = test_df5['Weather_Condition'].fillna(test_df5['Weather_Condition'].value_counts().index[0])

In [28]:
WC=pd.get_dummies(test_df5['Weather_Condition'])
test_df5 = pd.concat([test_df5, WC],axis=1)
test_df5 = test_df5.drop(columns=['Weather_Condition','date' , 'Time' , 'Selected'])

Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [29]:
X_test = test_df5.drop(columns=['ID'])
print(test_df5.shape)
# You should update/remove the next line once you change the features used for training
X_test = X_test[['Lat','Lng','Partly Cloudy','Distance(mi)','Crossing','Fair','Wind_Speed(mph)','Humidity(%)','Precipitation(in)','Stop','Amenity','Side','Year','Month','Wind_Chill(F)']]
print(X_test.shape)
y_test_predicted = classifier.predict(X_test)
print(y_test_predicted.shape)
test_df5['Severity'] = y_test_predicted

test_df.head()

(1601, 45)
(1601, 15)
(1601,)


,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,date,Time,Year,Month,Day
0,6407,37.786060,-122.390900,0,0.039,0,0,1,0,0,0,0,0,1,2016-04-04,19:20:31,2016,4,4
1,6408,37.769609,-122.415057,0,0.202,0,0,0,0,0,0,0,0,1,2020-10-28,11:51:00,2020,10,28
2,6409,37.807495,-122.476021,0,0.000,0,0,0,0,0,0,0,0,1,2019-09-09,07:36:45,2019,9,9
3,6410,37.761818,-122.405869,0,0.000,0,0,1,0,0,0,0,0,1,2019-08-06,15:46:25,2019,8,6
4,6411,37.732350,-122.414100,0,0.670,0,0,0,0,0,0,0,0,1,2018-10-17,09:54:58,2018,10,17


In [30]:
test_df5[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.